<h1 style="text-align:center;"><b>Laboratorio 6</b></h1>
<h3 style="text-align:center;">Alina Carías (22539), Ariela Mishaan (22052), Marcos Díaz (221102) </h3>

# Librerías

In [2]:
import pandas as pd
import json
import unicodedata
import string
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import networkx as nx

import community as community_louvain 
import collections

# ----- Análisis de tópicos con LDA (sklearn) -----
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 1. Carga de datos

## Tweets

In [16]:
with open("lab - datos redes/tioberny.txt", "r", encoding="utf-16") as f:
    tweetstxt = [json.loads(line) for line in f]

data = []
for t in tweetstxt:
    data.append({
        #"id": t.get("id"),
        "fecha": t.get("date"),
        "usuario": t["user"]["username"] if "user" in t else None,
        "nombre": t["user"]["displayname"] if "user" in t else None,
        "texto": t.get("rawContent"),
        "likes": t.get("likeCount"),
        "hashtags": t.get("hashtags"),
        #"mentioned": t.get("mentionedUsers"),
        "viewcount": t.get("viewCount"),
        "retweets": t.get("retweetCount"),
        "replies": t.get("replyCount"),
        "quotes": t.get("quoteCount")
        #"url": t.get("url")
    })

tweets = pd.DataFrame(data)
tweets.head()

,fecha,usuario,nombre,texto,likes,hashtags,viewcount,retweets,replies,quotes
0,2024-09-12 17:21:03+00:00,La_ReVoluZzion,The_ReVoluZZzioN 🫡,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",0,[],NaN,0,0,0
1,2024-09-12 15:27:20+00:00,XelaNewsGt,XelaNews,#URGENTE Lo que los medios #faferos no informa...,142,"[URGENTE, faferos, BernardoArévalo, NebajQuich...",5200.0,80,12,15
2,2024-09-12 17:20:25+00:00,M24095273,VIVAlafuenteDEtodaVIDA,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,0,[],NaN,0,0,0
3,2024-09-12 17:18:48+00:00,carlosalbesc,Carlos Alberto Escobar,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,0,[],NaN,0,0,0
4,2024-09-12 17:16:42+00:00,Brenda_AGN,Brenda Lari,El presidente @BArevalodeLeon y la vicepreside...,0,[],7.0,0,0,0


## Usuarios

In [12]:
user_data = {}
for t in tweetstxt:
    if "user" in t:
        u = t["user"]
        user_data[u["id"]] = {
            #"usuario_id": u["id"],
            "username": u.get("username"),
            "nombre": u.get("displayname"),
            "descripcion": u.get("rawDescription"),
            "creado_en": u.get("created"),
            "followers": u.get("followersCount"),
            "friends": u.get("friendsCount"),
            "tweets_totales": u.get("statusesCount"),
            "favourites": u.get("favouritesCount"),
            #"location": u.get("location"),
            #"url": u.get("url"),
            #"verified": u.get("verified"),
            #"profile_image": u.get("profileImageUrl"),
            #"profile_banner": u.get("profileBannerUrl")
        }

df_users = pd.DataFrame(list(user_data.values()))
df_users.head()

,username,nombre,descripcion,creado_en,followers,friends,tweets_totales,favourites
0,La_ReVoluZzion,The_ReVoluZZzioN 🫡,"¡Defensor de los defensores de los DDHH, PRO-L...",2021-09-07 02:11:21+00:00,2102,531,51984,143089
1,XelaNewsGt,XelaNews,Noticias sin injerencias.,2018-01-25 22:11:34+00:00,3462,359,2915,5719
2,M24095273,VIVAlafuenteDEtodaVIDA,CONCIENCIA TOTAL,2019-01-20 18:39:29+00:00,30,171,598,201
3,carlosalbesc,Carlos Alberto Escobar,,2014-11-17 13:58:10+00:00,11,218,4302,2817
4,Brenda_AGN,Brenda Lari,,2015-02-03 03:55:36+00:00,564,223,8707,4274


## Inspección inicial

In [17]:
tweets.info()
display(tweets.head())
tweets.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5019 entries, 0 to 5018
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fecha      5019 non-null   object 
 1   usuario    5019 non-null   object 
 2   nombre     5019 non-null   object 
 3   texto      5019 non-null   object 
 4   likes      5019 non-null   int64  
 5   hashtags   5019 non-null   object 
 6   viewcount  4938 non-null   float64
 7   retweets   5019 non-null   int64  
 8   replies    5019 non-null   int64  
 9   quotes     5019 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 392.2+ KB


,fecha,usuario,nombre,texto,likes,hashtags,viewcount,retweets,replies,quotes
0,2024-09-12 17:21:03+00:00,La_ReVoluZzion,The_ReVoluZZzioN 🫡,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",0,[],NaN,0,0,0
1,2024-09-12 15:27:20+00:00,XelaNewsGt,XelaNews,#URGENTE Lo que los medios #faferos no informa...,142,"[URGENTE, faferos, BernardoArévalo, NebajQuich...",5200.0,80,12,15
2,2024-09-12 17:20:25+00:00,M24095273,VIVAlafuenteDEtodaVIDA,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,0,[],NaN,0,0,0
3,2024-09-12 17:18:48+00:00,carlosalbesc,Carlos Alberto Escobar,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,0,[],NaN,0,0,0
4,2024-09-12 17:16:42+00:00,Brenda_AGN,Brenda Lari,El presidente @BArevalodeLeon y la vicepreside...,0,[],7.0,0,0,0


fecha         0
usuario       0
nombre        0
texto         0
likes         0
hashtags      0
viewcount    81
retweets      0
replies       0
quotes        0
dtype: int64

# 2. Limpieza y preprocesamiento

In [18]:
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

SPANISH_STOPWORDS = set(stopwords.words('spanish'))

In [20]:
# regex según acción por hacer
URL_RE = re.compile(r'https?://\S+|www\.\S+', re.IGNORECASE)
MENTION_RE = re.compile(r'@([A-Za-z0-9_]{1,15})')  # handle básico de X
HASHTAG_RE = re.compile(r'#(\w+)')
EMOJI_RE = re.compile(r'[\U00010000-\U0010ffff]', flags=re.UNICODE)

In [23]:
# funciones para parsear cada item de cada tweet. 
def normalize_username(u):
    if pd.isna(u):
        return u
    return str(u).strip().lower()

def clean_text(text: str):
    if pd.isna(text):
        return ''
    t = text
    t = URL_RE.sub(' ', t)
    t = EMOJI_RE.sub(' ', t)
    t = t.lower()
    t = unicodedata.normalize('NFKD', t)
    t = ''.join(ch for ch in t if not unicodedata.combining(ch))
    t = t.translate(str.maketrans('', '', string.punctuation))
    t = re.sub(r'\d+', ' ', t)
    tokens = [w for w in re.findall(r'\b\w+\b', t) if w not in SPANISH_STOPWORDS]
    return ' '.join(tokens)

# devuelve una lista de todas las menciones (@) encontradas en el tweet. 
def extract_mentions(text: str):
    return [m.lower() for m in MENTION_RE.findall(text or '')]

# devuelve una lista de todos los hashtags (#) encontradas en el tweet. 
def extract_hashtags(text: str):
    return [h.lower() for h in HASHTAG_RE.findall(text or '')]

# si empieza con "rt" lo marca como retweet. 
def is_retweet(text: str):
    return bool(re.match(r'^\s*rt\s*@', str(text).lower()))

# si empieza con "@usuario" lo marca como reply. 
def is_reply(text: str):
    return bool(re.match(r'^\s*@', str(text).lower()))

In [27]:
tweets['username'] = tweets['usuario'].map(normalize_username)
tweets['tweet_text_clean'] = tweets['texto'].map(clean_text)
tweets['mentions'] = tweets['texto'].map(extract_mentions)
tweets['hashtags'] = tweets['texto'].map(extract_hashtags)
tweets['is_retweet'] = tweets['texto'].map(is_retweet)
tweets['is_reply'] = tweets['texto'].map(is_reply)
tweets['fecha'] = pd.to_datetime(tweets['fecha'], errors='coerce')

# Eliminar duplicados por texto y usuario en el mismo instante
tweets = tweets.drop_duplicates(subset=['username','texto','fecha']).reset_index(drop=True)
display(tweets.head())

,fecha,usuario,nombre,texto,likes,hashtags,viewcount,retweets,replies,quotes,username,tweet_text_clean,mentions,is_retweet,is_reply
0,2024-09-12 17:21:03+00:00,La_ReVoluZzion,The_ReVoluZZzioN 🫡,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",0,[],NaN,0,0,0,la_revoluzzion,confirmado companeres impuesto usembassyguate ...,"[usembassyguate, 48cantonestoto, usaidguate, u...",False,False
1,2024-09-12 15:27:20+00:00,XelaNewsGt,XelaNews,#URGENTE Lo que los medios #faferos no informa...,142,"[urgente, faferos, bernardoarévalo, nebajquich...",5200.0,80,12,15,xelanewsgt,urgente medios faferos informaron ayer acerca ...,[],False,False
2,2024-09-12 17:20:25+00:00,M24095273,VIVAlafuenteDEtodaVIDA,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,0,[],NaN,0,0,0,m24095273,ivanduque barevalodeleon usaste pegasus espiar...,"[ivanduque, barevalodeleon]",False,True
3,2024-09-12 17:18:48+00:00,carlosalbesc,Carlos Alberto Escobar,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,0,[],NaN,0,0,0,carlosalbesc,ivanduque barevalodeleon entienden bien si est...,"[ivanduque, barevalodeleon]",False,True
4,2024-09-12 17:16:42+00:00,Brenda_AGN,Brenda Lari,El presidente @BArevalodeLeon y la vicepreside...,0,[],7.0,0,0,0,brenda_agn,presidente barevalodeleon vicepresidenta karin...,"[barevalodeleon, karinherreravp, agn_noticias,...",False,False
